In [ ]:
import pandas as pd
import numpy as np
import json
import os
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns

In [ ]:
sns.set_palette("colorblind")

In [ ]:
def ns_to_ms(ns):
  return ns / 1_000_000

## Data Loading

In [ ]:
pwd = "."
figs_dir = "./figs"
!mkdir -p figs

In [ ]:
def read_json(path):
  fp = open(path, 'r')
  nb_stats = json.load(fp)
  fp.close()
  return nb_stats

In [ ]:
modin_path = "stats_modin.json"
pandas_path = "stats_pd.json"

modin_stats = read_json(modin_path)
pandas_stats = read_json(pandas_path)

assert len(modin_stats['cells']) == len(pandas_stats['cells'])
d = {'Code': [], 'Pandas Time': [], 'Modin Time': []}
for i in range(len(pandas_stats['cells'])):
  pd_cell = pandas_stats['cells'][i]
  modin_cell = modin_stats['cells'][i]
  # The cell that imports is different for the pandas and modin versions
  if "import pandas" not in pd_cell['raw']:
    assert pd_cell['raw'] == modin_cell['raw']
  d['Code'].append(pd_cell['raw'])
  d['Pandas Time'].append(ns_to_ms(pd_cell['total-ns']))
  d['Modin Time'].append(ns_to_ms(modin_cell['total-ns']))

In [ ]:
df = pd.DataFrame(d)

## Stats

In [ ]:
abs_diff = df['Pandas Time'] - df['Modin Time']
rel_diff = df['Pandas Time'] / df['Modin Time']

In [ ]:
abs_diff[abs_diff > 200]

In [ ]:
rel_diff[rel_diff > 1.1]

In [ ]:
abs_diff.plot(kind='bar', xlabel='Notebook index', ylabel="Time (ms)")

In [ ]:
# If we want to check the code of a notebook
print(df.iloc[10]['Code'])